In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from scipy.stats import mode
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, classification_report, cohen_kappa_score, f1_score
from yellowbrick.classifier import ClassificationReport, ClassPredictionError, ConfusionMatrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from keras import Sequential
from keras.layers import Dense

In [3]:
df = pd.read_csv("../data/[CLEANED]kepler-data.csv")
df.drop(columns = ["Unnamed: 0"], inplace=True)
print(df.shape)
df.head()

(9110, 47)


,rowid,kepid,kepoi_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
ALL_COLUMNS = df.columns
ERROR_COLUMNS = [col for col in ALL_COLUMNS if "err" in col]
EXCLUDE = ["rowid", "kepid", "kepoi_name", "koi_score", "koi_disposition", "koi_pdisposition", "koi_tce_delivname", "koi_tce_plnt_num"] + ERROR_COLUMNS
TO_USE = list(set(ALL_COLUMNS) - set(EXCLUDE))

In [5]:
subset_df = df[df["koi_disposition"] != "CANDIDATE"]
print(subset_df.shape)
subset_df.head()

(6939, 47)


,rowid,kepid,kepoi_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [6]:
X = subset_df[TO_USE].values
X = StandardScaler().fit_transform(X)
y = subset_df["koi_disposition"].apply(lambda x: x=='CONFIRMED').astype(int).values

In [7]:
def performance(test, pred):
    conf_matrix = confusion_matrix(test, pred)
    f1 = f1_score(test, pred)
    report = classification_report(test, pred)
    accuracy = balanced_accuracy_score(test, pred)
    kappa = cohen_kappa_score(test, pred)
    print(f"F1 Score: {f1}")
    print(f"Kappa Score: {kappa}")
    print(f"Accuracy Score: {accuracy}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(report)

In [8]:
def createNNModel():
    model = Sequential([
                    Dense(256, input_dim=X.shape[1], activation = 'relu'),
                    Dense(128, activation = 'relu'),
                    Dense(128, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
                   ])
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [9]:
models = [
    AdaBoostClassifier(algorithm='SAMME',
                    base_estimator=DecisionTreeClassifier(max_depth=3),
                    learning_rate=0.9500000000000002, n_estimators=80,
                    random_state=0),
    RandomForestClassifier(n_jobs=-1, random_state=0),
    SVC(C=1.8500000000000008, random_state=0, tol=1),
    createNNModel()
]

model_names = ["adaboost", "rf", "svc", "nn"]

In [10]:
def trainEvaluate(model, model_name, fold, X_train, y_train, X_test, y_test):
    if model_name == "nn":
        model.fit(X_train, y_train, epochs=20, verbose=0)
    else:
        model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred = (pred >= 0.5).astype(int).flatten()
    #f1 = f1_score(y_test, pred)
    #print(f"F1 Score in fold {fold} = {f1}")
    return pred

def finalPrediction(preds):
    preds = np.asarray(preds)
    final = mode(preds)[0][0]
    return final

def crossValidation(K, models, model_names):
    kFold = KFold(n_splits=K, shuffle=True, random_state=0)
    f1_scores = list()
    k_ctr = 1
    pred_per_split = list()
    for train, test in kFold.split(X, y):
        pred_per_split.clear()
        for idx, model in enumerate(models):
            current_model_name = model_names[idx]
            preds = trainEvaluate(model, current_model_name, k_ctr, X[train], y[train], X[test], y[test])
            pred_per_split.append(preds)
        final_preds = finalPrediction(pred_per_split)
        current_f1 = f1_score(y[test], final_preds)
        print(f"F1 Score in fold {k_ctr} = {current_f1}")
        k_ctr+=1
        f1_scores.append(current_f1)
    print(f"Average {K}-Fold F1 Score = {np.mean(f1_scores)}\n")
    
    kFold = StratifiedKFold(n_splits=K, shuffle=True, random_state=0)
    f1_scores = list()
    k_ctr = 1
    pred_per_split = list()
    for train, test in kFold.split(X, y):
        pred_per_split.clear()
        for idx, model in enumerate(models):
            current_model_name = model_names[idx]
            preds = trainEvaluate(model, current_model_name, k_ctr, X[train], y[train], X[test], y[test])
            pred_per_split.append(preds)
        final_preds = finalPrediction(pred_per_split)
        current_f1 = f1_score(y[test], final_preds)
        print(f"F1 Score in fold {k_ctr} = {current_f1}")
        k_ctr+=1
        f1_scores.append(current_f1)
    print(f"Average Stratified {K}-Fold F1 Score = {np.mean(f1_scores)}")

In [11]:
crossValidation(10, models, model_names)

F1 Score in fold 1 = 0.9748283752860412
F1 Score in fold 2 = 0.9788235294117645
F1 Score in fold 3 = 0.9684684684684685
F1 Score in fold 4 = 0.9843400447427293
F1 Score in fold 5 = 0.9859154929577465
F1 Score in fold 6 = 0.989648033126294
F1 Score in fold 7 = 0.9855072463768116
F1 Score in fold 8 = 0.9935205183585313
F1 Score in fold 9 = 0.9834710743801652
F1 Score in fold 10 = 0.9840546697038726
Average 10-Fold F1 Score = 0.9828577452812425

F1 Score in fold 1 = 0.9890590809628009
F1 Score in fold 2 = 0.989010989010989
F1 Score in fold 3 = 0.9641255605381167
F1 Score in fold 4 = 0.973568281938326
F1 Score in fold 5 = 0.9801324503311258
F1 Score in fold 6 = 0.991304347826087
F1 Score in fold 7 = 0.9911894273127753
F1 Score in fold 8 = 0.9823788546255506
F1 Score in fold 9 = 0.986784140969163
F1 Score in fold 10 = 0.9800443458980044
Average Stratified 10-Fold F1 Score = 0.9827597479412941
